In [1]:
import os, shutil, zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\\Windows\\Fonts\\malgun.ttf").get_name()
plt.rc("font", family = font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

# import networkx as nx
# from networkx.drawing.nx_pydot import graphviz_layout

import random
import re
import json

# from urllib.request import urlopen
# from bs4 import BeautifulSoup

# 저녁추천(연관분석)

In [3]:
rules_d = pd.read_csv("content/rules_dinner2.csv",encoding="utf-8")

##### 저녁 추천 함수 정의
def recommend_dinner_conviction(input_dish, top_n=10):
    
    if input_dish not in list(rules_d['antecedents']):
        print(f"입력한 메뉴({input_dish})가 학습 데이터에 없습니다.")

#     print(f"입력한 메뉴({input_dish})를 통해 추천된 저녁 메뉴는 다음과 같습니다.")
    recommended_dinner = rules_d[rules_d['antecedents'] == input_dish].sort_values('conviction', ascending=False)[:top_n]
#     print(recommended_dinner['consequents'].tolist())

    assosiation_menu = list(recommended_dinner['consequents'].tolist())

    return assosiation_menu

# 저녁추천(군집화)

In [21]:
center_list1=pd.read_csv('content/centerlist.csv', encoding="utf-8")
center_list1=center_list1.rename(columns={'에너지(㎉)':'에너지(kcal)'})

cluster_items=[['하이라이스','표고버섯볶음밥','콩나물국밥','카레라이스','치즈김밥','채소볶음밥',
             '채소김밥','참치볶음밥','참치마요삼각김밥','참치김밥','짬뽕밥','짜장밥',
             '육회비빔밥','유부초밥','오징어덮밥','오므라이스','영양돌솥밥','알밥',
             '계란볶음밥','고추김밥','순대국밥','쇠고기주먹밥','쇠고기볶음밥','쇠고기김밥',
              '생선모듬초밥', '새우볶음밥','비빔밥','불고기덮밥','볶음밥', '모듬회덮밥',
              '돌솥비빔밥','돈까스김밥','닭고기덮밥','날치알김밥','낙지덮밥','김치볶음밥',
              '김치김밥','김밥'],
             ['호박고추장찌개','햄 김치찌개','해물매운탕','콩비지찌개','콩나물된장찌개','추어탕',
              '청국장찌개','참치김치찌개','조개된장찌개','오징어찌개','오리백숙','어묵김치찌개',
              '알탕','순두부찌개','순두부김치찌개','쇠고기전골','쇠고기된장찌개','쇠고기 샤브샤브',
              '버섯전골','버섯샤브샤브','두부애호박된장찌개','두부쇠고기고추장찌개','두부된장찌개',
              '두부고추장찌개','된장찌개','돼지고기찌개','돼지고기김치찌개','동태찌개','대구 매운탕',
              '닭볶음탕','달래된장찌개','냉이된장찌개','낙지전골','꽃게탕','꽁치김치찌개','김치전골',
              '갈치찌개'],
             ['함박스테이크','닭발구이','닭고기대파꼬치구이','굴비구이','갈치구이','가자미구이',
              '돼지고기산적','키조개구이','참치머리구이','조기구이','장어고추장구이','오리불고기',
              '안심스테이크','쇠불고기','쇠곱창구이','새우구이','삼치구이','뱅어포구이',
              '등심스테이크','돼지불고기'],
             ['칼국수','짬뽕','짜장면','쟁반국수','수제비','비빔냉면','비빔국수','볶음우동',
              '바지락칼국수','물냉면','라면','떡라면','달걀라면'],
             ['팔보채','죽순쇠고기볶음','잡채','유산슬','오징어볶음','쇠고기채소볶음','문어고추장볶음',
              '묵은지삼겹살볶음','라볶이','떡볶이','돼지곱창순대볶음','돼지고기김치볶음','닭발볶음',
              '낙지볶음'],
             ['뼈해장국','도가니탕','닭개장','꼬리곰탕','곰탕','새우탕','육개장','우거지곰탕','설렁탕',
              '선짓국(선지해장국)'],
             ['닭날개튀김','고등어튀김','간장양념치킨','간장양념닭다리튀김','후라이드치킨','치킨까스',
              '양념치킨','안심돈가스','생선까스','등심돈가스']]

col = ["식품명", "에너지(kcal)", "탄수화물(g)", "단백질(g)", "지방(g)", "총당류(g)", "나트륨(g)"]

##### 저녁추천(군집화) 함수 정의
def menu_cluster(remain_data, sangsu=20, columns=col) :
     
    # 잔여영양소와 클러스터(중심)간 차이  
    remain_df=pd.DataFrame(remain_data, index=col[1:]).T
    center_list2={'에너지(kcal)':[], '탄수화물(g)':[], '단백질(g)':[], '지방(g)':[],
              '총당류(g)':[], '나트륨(g)':[]}
    
    center_list2 = pd.DataFrame(center_list2)
    

    for column in columns[1:]:
        nutrition = []
        for i in range(len(center_list1)):
            nutrition.append(abs(center_list1[column][i] -  remain_df[column][0]))
        center_list2[column] = nutrition

    # 거리 계산
    center_list2["총합"] = center_list2.sum(axis=1)
    center_list2["Cluster"] = center_list1["Cluster"]
    
    # 클러스터 내 메뉴 개수 
    center_list2["메뉴개수"] = 1
    for i in range(len(cluster_items)):
        center_list2.loc[i,"메뉴개수"] = len(cluster_items[i])

    # 거리가 적은 순으로 정렬
    center_list2.sort_values("총합", inplace=True)
    center_list2.reset_index(drop=True, inplace=True)

    # 추천 메뉴 
    recom_list = []
    for cluster_num in range(0, len(center_list2)):
        if center_list2["메뉴개수"][cluster_num] == 1:
            recom_list.append(center_list2["Cluster"][cluster_num])
        else:
            recom_list.extend(cluster_items[int(center_list2["Cluster"][cluster_num])-1])

        if center_list2["메뉴개수"][0:cluster_num+1].sum() > sangsu:
            break
    
    return  recom_list

# 저녁추천(교집합+거리)

In [51]:
##### 식품영양DB 
def nutrient_db(columns=col) :
    nutrition_df = pd.read_excel("files/Nutri_food_DB.xlsx", sheet_name='20230215')
    nutrition_df=nutrition_df.rename(columns={'에너지(㎉)':'에너지(kcal)', '나트륨(㎎)':"나트륨(g)"})    
    nutrition_df=nutrition_df[columns][nutrition_df['지역 / 제조사']=='전국(대표)']
    
    for i in nutrition_df.index:
        nutrition_df['나트륨(g)'][i] = float(nutrition_df['나트륨(g)'][i])/1000
    
    return nutrition_df

#### 군집분석과 연관분석의 교집합 추출
def intersection_menu(list1, list2) :
    inter_menu= list(set(list1) & set(list2))
    return inter_menu

##### 저녁메뉴 추천
def dinner_recommedor(input_menu, remain_data, recom_num = 6, sansu=20, columns=col) : 
    
    # 식품영양db 
    nutrition_df=nutrient_db()
    
    # 군집분석과 연관분석의 교집합
    association = recommend_dinner_conviction(input_menu)
    cluster = menu_cluster(remain_data)
    
    inter_menu = intersection_menu(association , cluster)

    # 저녁메뉴 추천(6개) : 교집합 우선, 나머지는 거리가 짧은 순
    recom_menu = []
    diff_menu=[]

    if len(inter_menu) >= recom_num :
        recom_menu.extend(random.sample(inter_menu, recom_num))
    else :
        if len(inter_menu) == 0:
            pass 
        else : 
            recom_menu.extend(inter_menu)
            
        diff_menu=list(set(association) ^ set(cluster))

        diff_nutrient=[]
        
        remain_df=pd.DataFrame(remain_data, index=col[1:]).T
        
        for m in diff_menu : 
            term=nutrition_df[nutrition_df["식품명"]== m]
#             print(term)
            nutrient=[m]
            for column in columns[1:]:
#                 print(term[column].values[0])
#                 print('==============================')
#                 print(remain_df[column].values[0])
#                 print('==============================')
                nutrient.append(abs(term[column].values[0] - remain_df[column].values[0]))
            diff_nutrient.append(nutrient)
#             print(diff_nutrient)

        df_candidate=pd.DataFrame(diff_nutrient, columns=columns)
        df_candidate["총합"] = df_candidate.sum(axis=1)
        df_candidate.sort_values("총합", inplace=True)
        df_candidate.reset_index(drop=True, inplace=True)

        candi_num = (recom_num - len(inter_menu)) * 2
        candi_top = df_candidate.loc[:(candi_num-1)]
  
        candi_menu = candi_top["식품명"].values.tolist()

        rnd_menu=random.sample(candi_menu, int(candi_num/2))

        recom_menu.extend(rnd_menu)

    return recom_menu

# 시뮬레이션

In [52]:
##### data input
sex = '여자'
age = '12-14'
lunch= '순대국밥'
side= ['부추무침,김치전', '도토리묵무침', '메밀전병']

remain=[1159.08, 14.920000000000002, 20.260000000000005, 39.81666666666668, 47.87, 0.5166200000000001]

##### 저녁추천
# recommend_dinner_conviction(lunch)
# menu_cluster(remain)
dinner_recommedor(lunch, remain)

C:\Users\acorn\AppData\Local\Temp\ipykernel_17352\2727908099.py:61: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_candidate["총합"] = df_candidate.sum(axis=1)


['삼계탕', '삼겹살고추장구이', '닭구이', '오리탕', '미트볼 토마토 스파게티', '돼지갈비구이']